## Setup

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.output_result { max-width:90% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(linewidth=170)
np.set_printoptions(suppress=True)
np.random.seed(8)

from time import time, sleep
sleep(0.2)

C:\Users\raziel lab\AppData\Local\Temp\ipykernel_26368\1403808273.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Imports & Parameters

In [2]:
import pandas as pd

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)
random_state = 8

## Loading the data

In [3]:
freezers = pd.read_csv('C:/Users/raziel lab/PycharmProjects/ConsumptionModels/Appliances Algorithm/Freezers/freezers1.csv', encoding='utf8')
freezers.shape

(95, 29)

## Main

In [4]:
len(freezers['אינוונטר'].unique())

95

In [5]:
freezers['שם סטטוס'].unique()

array(['פעיל', 'מושאל'], dtype=object)

In [6]:
freezers = freezers[freezers['שם סטטוס'] == 'פעיל']
freezers['שם סטטוס'].unique()

array(['פעיל'], dtype=object)

In [7]:
print("Rows Number:",freezers.shape[0])

Rows Number: 94


In [8]:
freezers['שם קמפוס'].unique()

array(['קמפוס\xa0מרקוס', 'קמפוס\xa0שדה\xa0בוקר', 'פארק\xa0הייטק', 'מדעי\xa0הבריאות\xa0סורוקה', 'קרית\xa0ברגמן'], dtype=object)

In [9]:
freezers = freezers[(freezers['שם קמפוס'] == 'קמפוס מרקוס') | (pd.isnull(freezers['שם קמפוס']))]
freezers['שם קמפוס'].unique()

array(['קמפוס\xa0מרקוס'], dtype=object)

In [10]:
print("Rows Number:", freezers.shape[0])

Rows Number: 64


In [11]:
freezers.drop(['אינוונטר','סטטוס','שם סטטוס','תאריך עדכון אחרון','פריט','מחיר כניסה בדולר','הזמנה','סריאלי','תז בעל מצאי',' שם תז בעל מצאי','שם בעל מצאי','מחיר ידני','קוד סימון','תז שואל','פטור מותנה','מנוהל תמיד','קמפוס','שם קמפוס','קומה','חדר','הערות לפריט אומנות'], axis=1, inplace=True)
freezers.reset_index(drop=True, inplace=True)
freezers.head()

,תאריך קבלה,"מחיר כניסה בש""ח",תיאור האינוונטר,הערות לשורה מההזמנה,מחלקה,שם מחלקה,בניין,שם בניין
0,04/08/2019,"38,393.43",מקפיא עמוק,ExF40086 Revco ExF Ultra-Low Temperature Uprig...,5104,מדעי החיים,40,"בנין המחלקה למדעי החיים ע""ש טומן - 40"
1,29/04/2015,"1,734.60",מקפיא 4 מגירות דגם GP 1366,מקפיא ליבהר דגם 4 1376 מגרות,5104,מדעי החיים,40,"בנין המחלקה למדעי החיים ע""ש טומן - 40"
2,29/04/2015,"1,734.60",מקפיא 4 מגירות דגם GP 1366,מקפיא ליבהר דגם 4 1376 מגרות,5104,מדעי החיים,40,"בנין המחלקה למדעי החיים ע""ש טומן - 40"
3,29/04/2015,"1,734.60",מקפיא 4 מגירות דגם GP 1366,מקפיא ליבהר 1376. מקפיא ליהבר דגם 1376,5104,מדעי החיים,40,"בנין המחלקה למדעי החיים ע""ש טומן - 40"
4,29/04/2015,"1,699.93","מקפיא דגם, Liebherr GP1376",מקפיא ליבר 1376,5104,מדעי החיים,40,"בנין המחלקה למדעי החיים ע""ש טומן - 40"


In [12]:
freezers.rename(columns = {'תאריך קבלה':'operating_date',
                           'מחיר כניסה בש"ח':'price',
                           'תיאור האינוונטר':'description',
                           'הערות לשורה מההזמנה':'comment',
                           'מחלקה':'department_number',
                           'שם מחלקה':'department_name',
                           'בניין':'building_number',
                           'שם בניין':'building_name'}, inplace=True)
freezers.head()

,operating_date,price,description,comment,department_number,department_name,building_number,building_name
0,04/08/2019,"38,393.43",מקפיא עמוק,ExF40086 Revco ExF Ultra-Low Temperature Uprig...,5104,מדעי החיים,40,"בנין המחלקה למדעי החיים ע""ש טומן - 40"
1,29/04/2015,"1,734.60",מקפיא 4 מגירות דגם GP 1366,מקפיא ליבהר דגם 4 1376 מגרות,5104,מדעי החיים,40,"בנין המחלקה למדעי החיים ע""ש טומן - 40"
2,29/04/2015,"1,734.60",מקפיא 4 מגירות דגם GP 1366,מקפיא ליבהר דגם 4 1376 מגרות,5104,מדעי החיים,40,"בנין המחלקה למדעי החיים ע""ש טומן - 40"
3,29/04/2015,"1,734.60",מקפיא 4 מגירות דגם GP 1366,מקפיא ליבהר 1376. מקפיא ליהבר דגם 1376,5104,מדעי החיים,40,"בנין המחלקה למדעי החיים ע""ש טומן - 40"
4,29/04/2015,"1,699.93","מקפיא דגם, Liebherr GP1376",מקפיא ליבר 1376,5104,מדעי החיים,40,"בנין המחלקה למדעי החיים ע""ש טומן - 40"


In [13]:
freezers.shape

(64, 8)

In [14]:
domestic_df = pd.DataFrame(columns=freezers.columns)
industrial_df = pd.DataFrame(columns=freezers.columns)

In [15]:
description_col = freezers.columns.get_loc('description')

for i in range (freezers['description'].shape[0]):
    
    if 'עמוק' in freezers.iloc[i,description_col]:
        freezers.iloc[i,description_col] = 'Industrial'
    if 'תעשייתי' in freezers.iloc[i,description_col]:
        freezers.iloc[i,description_col] = 'Industrial'

In [16]:
industrial_rows = freezers[freezers['description'] == 'Industrial']

industrial_df = industrial_df.append(industrial_rows, ignore_index = True)

freezers.drop(industrial_rows.index, inplace = True)

domestic_df = domestic_df.append(freezers, ignore_index = True)

industrial_df.drop(['description', 'comment'], axis=1, inplace = True)
domestic_df.drop(['description', 'comment'], axis=1, inplace = True)

In [17]:
print ('Industrial Building Numbers:', industrial_df['building_number'].unique())
print()
industrial_df = industrial_df[industrial_df['building_number'] != 'M6']
industrial_df.reset_index(drop=True, inplace=True)
industrial_df.drop(['price','department_number', 'department_name', 'building_name'], axis=1, inplace=True)

print ('Domestic Building Numbers:', domestic_df['building_number'].unique())
domestic_df = domestic_df[domestic_df['building_number'] != 'M6']
domestic_df = domestic_df[domestic_df['building_number'] != 'M7']
domestic_df.reset_index(drop=True, inplace=True)
domestic_df.drop(['price','department_number', 'department_name', 'building_name'], axis=1, inplace=True)

Industrial Building Numbers: ['40' 'M6' '38' '39' '29']

Domestic Building Numbers: ['40' 'M6' '42' '39' '29' '58' '33' '97' 'M7' '51' '38' '59' '41' '65']


In [18]:
industrial_df['operating_date'] = pd.to_datetime(industrial_df['operating_date'])
domestic_df['operating_date'] = pd.to_datetime(domestic_df['operating_date'])

In [19]:
months=12
for i in range(1, months+1):

    string1 = '2019-'
    if i != 12:
        string2 = str(i+1)
        final_date = string1+string2
    else:        
        string2 = str(i)
        string3 = '-31'
        final_date = string1+string2+string3
    final_date = pd.to_datetime(final_date)

    industrial_final = industrial_df[~(industrial_df['operating_date'] >= final_date)]
    industrial_final = industrial_final.drop(['operating_date'], axis = 1, inplace=False)

    domestic_final = domestic_df[~(domestic_df['operating_date'] >= final_date)]
    domestic_final = domestic_final.drop(['operating_date'], axis = 1, inplace=False)

    industrial_final = industrial_final.groupby(['building_number']).size().reset_index(name='f_industrial')
    domestic_final = domestic_final.groupby(['building_number']).size().reset_index(name='f_domestic')

    total_freezers = pd.merge(industrial_final, domestic_final, how='outer', on='building_number')

    total_freezers = total_freezers.fillna(0)
    total_freezers.sort_values('building_number', axis=0, ascending=True, inplace=True, ignore_index=True)

    file_name = 'total_freezers_'+string1+str(i)+'.csv'
    total_freezers.to_csv(file_name, index=False, encoding="utf-8")